In [4]:
import gzip
import pandas as pd
import numpy as np
import re
import shutil
from warcio.archiveiterator import ArchiveIterator

# Open the .warc.gz file
with gzip.open("../data/warcs/sample.warc.gz", "rb") as f:
    # Read the contents of the file
    contents = f.read()

# Do something with the contents of the file
len(contents)

34854850

In [5]:
type(contents)
# Convert to string 
contents = contents.decode("latin-1")

In [6]:
split_list = re.split("WARC\/\d\.\d", contents)

In [12]:
split_list[3]

'\r\nWARC-Type: response\r\nWARC-Date: 2012-02-10T22:46:49Z\r\nWARC-TREC-ID: clueweb12-0000tw-00-00001\r\nWARC-IP-Address: 100.42.48.208\r\nWARC-Payload-Digest: sha1:437VOUPPSTZIBWRBB2IYFZ6JNEFBAWHQ\r\nWARC-Target-URI: http://tsawer.net/2012/02/10/le-canigou-dans-la-brume/\r\nWARC-Record-ID: <urn:uuid:3f40427b-b986-419e-b76f-40480020d933>\r\nContent-Type: application/http; msgtype=response\r\nContent-Length: 23000\r\n\r\nHTTP/1.1 200 OK\r\nDate: Fri, 10 Feb 2012 22:46:47 GMT\r\nServer: Apache/2.2.21 (Unix) mod_ssl/2.2.21 OpenSSL/0.9.8e-fips-rhel5 mod_auth_passthrough/2.1 mod_bwlimited/1.4 FrontPage/5.0.2.2635 mod_jk/1.2.32\r\nExpires: Thu, 19 Nov 1981 08:52:00 GMT\r\nCache-Control: no-store, no-cache, must-revalidate, post-check=0, pre-check=0\r\nPragma: no-cache\r\nX-Pingback: http://tsawer.net/wpaggregator/xmlrpc.php\r\nLink: <http://tsawer.net/?p=160052>; rel=shortlink\r\nVary: Accept-Encoding,User-Agent\r\nConnection: close\r\nContent-Type: text/html; charset=UTF-8\r\n\r\n<!DOCTYPE

In [ ]:
split_li

In [13]:
def read_warc(fname_warc_file):

    # put docs and ids in arrays to use in dataframe
    html_list = np.array([])
    warc_trec_id_list = np.array([])
    doc_and_id_df = pd.DataFrame()

    # read warc file and split by "WARC/number"
    with gzip.open(fname_warc_file, "rb") as stream:
        print("reading warc file...")

        read = stream.read()
        # convert read to string
        read = read.decode("latin-1")
        split_list = re.split("WARC\/\d\.\d", read)

        # loop through the data and check for trec id's and html-tags
        for warc_data in split_list:
            if len(warc_data) <= 1:
                continue
            if "WARC-TREC-ID" not in warc_data:
                continue
            if "<html>" not in warc_data:
                continue
            if "</html>" not in warc_data:
                continue

            # print('ITS A VALID WARC DATA!?', warc_data)

            one_warc_data = warc_data

            # get html document
            html_index_1 = one_warc_data.index("<html")
            html_index_2 = one_warc_data.index("</html>") + 7
            html_doc = one_warc_data[html_index_1:html_index_2]
            html_list = np.append(html_list, html_doc)
            # print('HTML DOC', html_doc)

            # get warc trec id
            warc_trec_id = re.search("WARC-TREC-ID: [\d\w\-\ ]+", one_warc_data)
            warc_trec_id = warc_trec_id.group(0)[len("WARC-TREC-ID: ") :]
            warc_trec_id_list = np.append(warc_trec_id_list, warc_trec_id)
            # print('WARC-TREC-ID', warc_trec_id)

    # invalid warc data, throw error
    if len(html_list) < 1 or len(warc_trec_id_list) < 1:
        raise Exception("No valid warc data found! Missing WARC-TREC-ID or html-tags")

    # if data is found, save to dataframe
    print("warc list", warc_trec_id_list)

    doc_and_id_df["HTML_DOC"] = html_list
    doc_and_id_df["WARC-TREC-ID"] = warc_trec_id_list

    return doc_and_id_df


In [14]:
read_warc("../data/warcs/sample.warc.gz")

reading warc file...
warc list ['clueweb12-0000tw-00-00006' 'clueweb12-0000tw-00-00043'
 'clueweb12-0000tw-00-00044' 'clueweb12-0000tw-00-00101'
 'clueweb12-0000tw-00-00129' 'clueweb12-0000tw-00-00130'
 'clueweb12-0000tw-00-00141' 'clueweb12-0000tw-00-00160'
 'clueweb12-0000tw-00-00161' 'clueweb12-0000tw-00-00162'
 'clueweb12-0000tw-00-00199' 'clueweb12-0000tw-00-00205'
 'clueweb12-0000tw-00-00210' 'clueweb12-0000tw-00-00212'
 'clueweb12-0000tw-00-00214' 'clueweb12-0000tw-00-00829'
 'clueweb12-0000tw-00-00833' 'clueweb12-0000tw-00-00834'
 'clueweb12-0000tw-00-00835' 'clueweb12-0000tw-00-00836'
 'clueweb12-0000tw-00-00837' 'clueweb12-0000tw-00-00843'
 'clueweb12-0000tw-00-00844' 'clueweb12-0000tw-00-00877'
 'clueweb12-0000tw-00-00879' 'clueweb12-0000tw-00-00880'
 'clueweb12-0000tw-00-00881' 'clueweb12-0000tw-00-00941'
 'clueweb12-0000tw-00-00942' 'clueweb12-0000tw-00-00943'
 'clueweb12-0000tw-00-00944' 'clueweb12-0000tw-00-00946']


,HTML_DOC,WARC-TREC-ID
0,"<html>\r\n\r\n<head>\r\n<meta http-equiv=""Cont...",clueweb12-0000tw-00-00006
1,"<html>\n\n<head>\n\n<style type=""text/css"">\n\...",clueweb12-0000tw-00-00043
2,"<html>\n<head>\n<style type=""text/css"">\n#bott...",clueweb12-0000tw-00-00044
3,<html>\n<head>\n\n<title>DHP Concerts Bristol ...,clueweb12-0000tw-00-00101
4,<html>\n<head>\n <title>12 Tourist Spots in ...,clueweb12-0000tw-00-00129
5,<html>\r\n<head>\r\n<title>Saving Money Buying...,clueweb12-0000tw-00-00130
6,<html>\n<head>\n <title>Freelance Web Desig...,clueweb12-0000tw-00-00141
7,"<html>\n<head>\n<meta http-equiv=""Content-Type...",clueweb12-0000tw-00-00160
8,"<html>\n<head>\n <meta http-equiv=""Content-...",clueweb12-0000tw-00-00161
9,"<html>\n<head>\n <meta http-equiv=""Content-...",clueweb12-0000tw-00-00162
